# Code Along: Reactions Table

## Start

Setting up.

In [3]:
import polars as pl
import polars.selectors as cs
from great_tables import GT, md
from great_tables.data import reactions

reactions

,cmpd_name,cmpd_mwt,cmpd_formula,cmpd_type,cmpd_smiles,cmpd_inchi,cmpd_inchikey,OH_k298,OH_uncert,OH_u_fac,...,NO3_t_low,NO3_t_high,Cl_k298,Cl_uncert,Cl_u_fac,Cl_A,Cl_B,Cl_n,Cl_t_low,Cl_t_high
0,methane,16.04,CH4,normal alkane,C,InChI=1S/CH4/h1H4,VNWKTOKETHGBQD-UHFFFAOYSA-N,6.360000e-15,0.1,NaN,...,NaN,NaN,1.000000e-13,0.15,NaN,6.600000e-12,1240.0,NaN,200.0,300.0
1,formaldehyde,30.03,CH2O,aldehyde,C=O,InChI=1S/CH2O/c1-2/h1H2,WSFSSNUMVMOOMR-UHFFFAOYSA-N,8.500000e-12,0.2,NaN,...,NaN,NaN,7.200000e-11,0.15,NaN,8.100000e-11,34.0,NaN,200.0,500.0
2,methanol,32.04,CH4O,alcohol or glycol,CO,"InChI=1S/CH4O/c1-2/h2H,1H3",OKKJLVBELUTLKV-UHFFFAOYSA-N,8.780000e-13,0.1,NaN,...,250.0,370.0,5.100000e-11,0.20,NaN,5.100000e-11,0.0,NaN,225.0,950.0
3,fluoromethane,34.03,CH3F,haloalkane (separated),CF,InChI=1S/CH3F/c1-2/h1H3,NBVXSUQYWXRMNV-UHFFFAOYSA-N,1.970000e-14,0.1,NaN,...,NaN,NaN,3.600000e-13,NaN,1.4,4.900000e-12,781.0,NaN,200.0,300.0
4,formic acid,46.03,CH2O2,carboxylic acid,OC=O,"InChI=1S/CH2O2/c2-1-3/h1H,(H,2,3)",BDAGIHXWWSANSR-UHFFFAOYSA-N,4.500000e-13,NaN,1.4,...,NaN,NaN,1.900000e-13,NaN,1.4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,gamma-heptalactone,128.17,C7H12O2,ether,CCCC1CCC(=O)O1,"InChI=1S/C7H12O2/c1-2-3-6-4-5-7(8)9-6/h6H,2-5H...",VLSVVMPLPMNWBH-UHFFFAOYSA-N,8.220000e-12,0.4,NaN,...,NaN,NaN,1.690000e-10,0.20,NaN,NaN,NaN,NaN,NaN,NaN
1679,"1,2-epoxyhexane",100.16,C6H12O,ether,CCCCC1CO1,"InChI=1S/C6H12O/c1-2-3-4-6-5-7-6/h6H,2-5H2,1H3",WHNBDXQTMPYBAT-UHFFFAOYSA-N,5.760000e-12,0.2,NaN,...,NaN,NaN,1.640000e-10,0.15,NaN,NaN,NaN,NaN,NaN,NaN
1680,"trans-2,3-epoxybutane",72.11,C4H8O,ether,C[C@H]1O[C@@H]1C,"InChI=1S/C4H8O/c1-3-4(2)5-3/h3-4H,1-2H3/t3-,4-...",PQXKWPLDPFFDJP-QWWZWVQMSA-N,1.870000e-12,0.2,NaN,...,NaN,NaN,6.690000e-11,0.15,NaN,NaN,NaN,NaN,NaN,NaN
1681,2-methyl-1-nitroprop-1-ene,101.10,C4H7NO2,aromatic halogen-containing hydrocarbon (multi...,CC(=C[N+](=O)[O-])C,"InChI=1S/C4H7NO2/c1-4(2)3-5(6)7/h3H,1-2H3",VXMMUDFDTWWSQT-UHFFFAOYSA-N,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Transforming the data.

In [2]:
reactions_mini = (
    pl.from_pandas(reactions)
    .filter(pl.col("cmpd_type") == "mercaptan")
    .select([
        "cmpd_name",
        "cmpd_formula",
        cs.ends_with("k298")
    ])
    .with_columns(
        cmpd_formula=pl.concat_str(
            "%" + pl.col("cmpd_formula") + "%"
        )
    )
)

reactions_mini

cmpd_name,cmpd_formula,OH_k298,O3_k298,NO3_k298,Cl_k298
str,str,f64,f64,f64,f64
"""methanethiol""","""%CH4S%""",3.5000e-11,null,9.2000e-13,2.0000e-10
"""ethanethiol""","""%C2H6S%""",4.5000e-11,null,1.2100e-12,1.7500e-10
"""propanethiol""","""%C3H8S%""",5.3000e-11,null,null,2.1400e-10
"""2-propanethiol""","""%C3H8S%""",3.9000e-11,null,null,2.7000e-10
"""1-butanethiol""","""%C4H10S%""",5.6000e-11,null,null,null
…,…,…,…,…,…
"""2-butanethiol""","""%C4H10S%""",3.8000e-11,null,null,1.6500e-10
"""t-butylsulfide""","""%C4H10S%""",2.9000e-11,null,null,null
"""2-methylbutanethiol""","""%C5H12S%""",5.2000e-11,null,null,null


Let's get the data into Great Tables. Plus let's make a stub.

In [5]:
gt_tbl_1 = GT(reactions_mini, rowname_col="cmpd_name") 

gt_tbl_1

,cmpd_formula,OH_k298,O3_k298,NO3_k298,Cl_k298
methanethiol,%CH4S%,3.5e-11,None,9.2e-13,2e-10
ethanethiol,%C2H6S%,4.5e-11,None,1.21e-12,1.75e-10
propanethiol,%C3H8S%,5.3e-11,None,None,2.14e-10
2-propanethiol,%C3H8S%,3.9e-11,None,None,2.7e-10
1-butanethiol,%C4H10S%,5.6e-11,None,None,None
2-methyl-1-propanethiol,%C4H10S%,4.6e-11,None,None,None
2-butanethiol,%C4H10S%,3.8e-11,None,None,1.65e-10
t-butylsulfide,%C4H10S%,2.9e-11,None,None,None
2-methylbutanethiol,%C5H12S%,5.2e-11,None,None,None
n-pentanethiol,%C5H12S%,None,None,None,1.97e-10


## Introducing Units Notation

Add a title to the table to explain contents.

In [6]:
gt_tbl_2 =  (
    gt_tbl_1
    .tab_header(title=md("Gas-Phase reactions of selected **mercaptan** compounds"))
)

gt_tbl_2

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='cmpd_formula', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH_k298', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.default: 1>, column_label='O3_k298', column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label='NO3_k298', column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl_k298', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table',

Group numerical columns with a spanner. Use this for the label:
`"Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}"`

In [7]:
gt_tbl_3 =  (
     gt_tbl_2
    .tab_spanner(
        label = "Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}",
        columns = cs.ends_with("k298")
    )
)

gt_tbl_3

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='cmpd_formula', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH_k298', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.default: 1>, column_label='O3_k298', column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label='NO3_k298', column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl_k298', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_backgro

Change Column labels for readability. The column names are:
`cmpd_formula`, `OH_k298`, `O3_k298`, `NO3_k298`, and `Cl_k298`.

In [13]:
gt_tbl_4 =  (
    gt_tbl_3
    .cols_label(
        cmpd_formula="",
        OH_k298= "OH",
        O3_k298 = "{{O_3}}" ,
        NO3_k298 = "{{NO_3}}",
        Cl_k298 = "Cl"
    )
)

gt_tbl_4

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=Tru

## Formatting for Science with fmt_units() and fmt_scientific()

Format the chemical formulas to make them look better.

In [14]:
gt_tbl_5 =  (
     gt_tbl_4
    .fmt_units(columns= "cmpd_formula")
)

gt_tbl_5

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002306CC3C050>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='tabl

Format the numeric values.

In [15]:
gt_tbl_6 =  (
     gt_tbl_5
    .fmt_scientific(columns=cs.ends_with("k298"))
)

gt_tbl_6

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002306CC3C050>, <great_tables._gt_data.FormatInfo object at 0x000002306CB2F410>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', 

Replace the `None` values.

In [16]:
gt_tbl_7 =  (
     gt_tbl_6
    .sub_missing(columns=cs.ends_with("k298"))
)

gt_tbl_7

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002306CC3C050>, <great_tables._gt_data.FormatInfo object at 0x000002306CB2F410>], _substitutions=[<great_tables._gt_data.FormatInfo object at 0x000002306CC41910>], _options=Options(ta

Hide redundant columns.

In [17]:
gt_tbl_8 =  (
     gt_tbl_7
    .cols_hide(columns="O3_k298")
)

gt_tbl_8

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.hidden: 4>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002306CC3C050>, <great_tables._gt_data.FormatInfo object at 0x000002306CB2F410>], _substitutions=[<great_tables._gt_data.FormatInfo object at 0x000002306CC41910>], _options=Options(tab

## Using the opt_*() Methods for Quick Styling

Use theming to style quickly.

In [21]:
gt_tbl_9 = (
     gt_tbl_8
    .opt_stylize(color="red") # default is blue, it also has style as argument, which ranges from 1 to 6
)

gt_tbl_9

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.hidden: 4>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002306CC3C050>, <great_tables._gt_data.FormatInfo object at 0x000002306CB2F410>], _substitutions=[<great_tables._gt_data.FormatInfo object at 0x000002306CC41910>], _options=Options(tab

Don't use the default font.

In [22]:
from great_tables import system_fonts

gt_tbl_10 = (
    gt_tbl_9
    .opt_table_font(font=system_fonts(name="humanist"))
)

gt_tbl_10

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.hidden: 4>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002306CC3C050>, <great_tables._gt_data.FormatInfo object at 0x000002306CB2F410>], _substitutions=[<great_tables._gt_data.FormatInfo object at 0x000002306CC41910>], _options=Options(tab

Make more space between neighboring values.

In [23]:
gt_tbl_11 = (
    gt_tbl_10
    .opt_horizontal_padding(scale= 3)
)

gt_tbl_11

GT(_tbl_data=shape: (11, 6)
┌─────────────────────┬──────────────┬────────────┬─────────┬────────────┬────────────┐
│ cmpd_name           ┆ cmpd_formula ┆ OH_k298    ┆ O3_k298 ┆ NO3_k298   ┆ Cl_k298    │
│ ---                 ┆ ---          ┆ ---        ┆ ---     ┆ ---        ┆ ---        │
│ str                 ┆ str          ┆ f64        ┆ f64     ┆ f64        ┆ f64        │
╞═════════════════════╪══════════════╪════════════╪═════════╪════════════╪════════════╡
│ methanethiol        ┆ %CH4S%       ┆ 3.5000e-11 ┆ null    ┆ 9.2000e-13 ┆ 2.0000e-10 │
│ ethanethiol         ┆ %C2H6S%      ┆ 4.5000e-11 ┆ null    ┆ 1.2100e-12 ┆ 1.7500e-10 │
│ propanethiol        ┆ %C3H8S%      ┆ 5.3000e-11 ┆ null    ┆ null       ┆ 2.1400e-10 │
│ 2-propanethiol      ┆ %C3H8S%      ┆ 3.9000e-11 ┆ null    ┆ null       ┆ 2.7000e-10 │
│ 1-butanethiol       ┆ %C4H10S%     ┆ 5.6000e-11 ┆ null    ┆ null       ┆ null       │
│ …                   ┆ …            ┆ …          ┆ …       ┆ …          ┆ …          │
│ 2-butanethiol       ┆ %C4H10S%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ 1.6500e-10 │
│ t-butylsulfide      ┆ %C4H10S%     ┆ 2.9000e-11 ┆ null    ┆ null       ┆ null       │
│ 2-methylbutanethiol ┆ %C5H12S%     ┆ 5.2000e-11 ┆ null    ┆ null       ┆ null       │
│ n-pentanethiol      ┆ %C5H12S%     ┆ null       ┆ null    ┆ null       ┆ 1.9700e-10 │
│ 1,2-ethanedithiol   ┆ %C2H6S2%     ┆ 3.8000e-11 ┆ null    ┆ null       ┆ null       │
└─────────────────────┴──────────────┴────────────┴─────────┴────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002306B017BD0>, _boxhead=Boxhead([ColInfo(var='cmpd_name', type=<ColInfoTypeEnum.stub: 2>, column_label='cmpd_name', column_align='left', column_width=None), ColInfo(var='cmpd_formula', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='OH_k298', type=<ColInfoTypeEnum.default: 1>, column_label='OH', column_align='right', column_width=None), ColInfo(var='O3_k298', type=<ColInfoTypeEnum.hidden: 4>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='O_3', unit='O', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='NO3_k298', type=<ColInfoTypeEnum.default: 1>, column_label=UnitStr(['', UnitDefinitionList([UnitDefinition(token='NO_3', unit='NO', unit_subscript='3', exponent=None, sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), column_align='right', column_width=None), ColInfo(var='Cl_k298', type=<ColInfoTypeEnum.default: 1>, column_label='Cl', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002306B087B90>, _spanners=Spanners([SpannerInfo(spanner_id='Reaction Rate Constant (298 K),<br>{{cm^3 molecules^–1 s^–1}}', spanner_level=0, spanner_label=UnitStr(['Reaction Rate Constant (298 K),<br>', UnitDefinitionList([UnitDefinition(token='cm^3', unit='cm', unit_subscript=None, exponent='3', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='molecules^–1', unit='molecules', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None), UnitDefinition(token='s^–1', unit='s', unit_subscript=None, exponent='–1', sub_super_overstrike=False, chemical_formula=False, built=None)]), '']), spanner_units=None, spanner_pattern=None, vars=['OH_k298', 'O3_k298', 'NO3_k298', 'Cl_k298'], built=None)]), _heading=Heading(title=Md(text='Gas-Phase reactions of selected **mercaptan** compounds'), subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002306CA85B50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002306CC3C050>, <great_tables._gt_data.FormatInfo object at 0x000002306CB2F410>], _substitutions=[<great_tables._gt_data.FormatInfo object at 0x000002306CC41910>], _options=Options(tab